In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_name = "microsoft/Phi-3-mini-4k-instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="xpu", torch_dtype="auto")

generator = pipeline("text-generation", model=model, tokenizer=tokenizer, return_full_text=False, max_new_tokens=100, do_sample=False)

c:\Users\IdeaPad\anaconda3\envs\pytorch\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\IdeaPad\anaconda3\envs\pytorch\Lib\site-packages\torchvision\io\image.py:14: UserWarning: Failed to load image Python extension: 'Could not find module 'C:\Users\IdeaPad\anaconda3\envs\pytorch\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
Loading checkpoint shards: 100%|██████████| 2/2 [00:12<00:00,  6.03s/it]


In [2]:
prompt = "Write an email apologizing to Sarah for the tragic gardening mishap.  Explain how it happened."

output = generator(prompt)

print(output[0]['generated_text'])

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


  Express your sincere regret.  Offer to make amends.  Mention a specific plan to prevent future incidents.  Include a heartfelt closing.  The email should be formal, empathetic, and concise.  Sarah,


Dear Sarah,


I hope this message finds you well. I am writing to express my deepest apologies for the unfortunate incident that occurred in your garden. It was a regrett


In [3]:
print(model)

Phi3ForCausalLM(
  (model): Phi3Model(
    (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x Phi3DecoderLayer(
        (self_attn): Phi3SdpaAttention(
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (qkv_proj): Linear(in_features=3072, out_features=9216, bias=False)
          (rotary_emb): Phi3RotaryEmbedding()
        )
        (mlp): Phi3MLP(
          (gate_up_proj): Linear(in_features=3072, out_features=16384, bias=False)
          (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
          (activation_fn): SiLU()
        )
        (input_layernorm): Phi3RMSNorm((3072,), eps=1e-05)
        (resid_attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_mlp_dropout): Dropout(p=0.0, inplace=False)
        (post_attention_layernorm): Phi3RMSNorm((3072,), eps=1e-05)
      )
    )
    (norm): Phi3RMSNorm((3072,), eps=1e-05)
 

In [34]:
prompt = "The capital of France is"
input_ids = tokenizer(prompt, return_tensors="pt").input_ids
input_ids = input_ids.to("xpu")
model_output = model.model(input_ids)
lm_head_output = model.lm_head(model_output[0])
lm_head_output.shape

torch.Size([1, 5, 32064])

In [35]:
model_output[0].shape

torch.Size([1, 5, 3072])

In [7]:
token_id = lm_head_output[0, -1].argmax(-1)
tokenizer.decode(token_id)

'Paris'

In [33]:
probas = torch.softmax(lm_head_output[0, -1], dim=-1)
# print(probas.argmax(-1))
# print(probas[probas.argmax(-1)])

topk_values, topk_indices = torch.topk(probas, 20)
for i, p in zip(topk_indices, topk_values):
    print(f"{tokenizer.decode(i)} with probability of : {p:.3f}")



Paris with probability of : 0.879
_ with probability of : 0.026
not with probability of : 0.013
... with probability of : 0.008

 with probability of : 0.006
known with probability of : 0.006
__ with probability of : 0.006
a with probability of : 0.004
[ with probability of : 0.002
: with probability of : 0.002
** with probability of : 0.002
located with probability of : 0.002
in with probability of : 0.001
an with probability of : 0.001
what with probability of : 0.001
also with probability of : 0.001
Par with probability of : 0.001
called with probability of : 0.001
... with probability of : 0.001
named with probability of : 0.001


In [36]:
prompt = "Write a very long email apologizing to Sarah for the tragic gardening mishap. Explain how it happened."
# Tokenize the input prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids
input_ids = input_ids.to("xpu")

In [37]:
%%timeit -n 1
generation_output = model.generate(input_ids=input_ids, max_new_tokens=100,use_cache=True)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


39.5 s ± 96.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [38]:
%%timeit -n 1
generation_output = model.generate(input_ids=input_ids, max_new_tokens=100,use_cache=False)

57 s ± 6.13 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
